# Equivalence Checking of Parameterized Quantum Circuits

论文提出了一种基于ZX演算的含参量子线路等价性验证方法，本文尝试在MindSpore Quantum架构中复现论文所提到的量子线路等价方法。

In [1]:
# 引入相关库
from mindquantum.core.circuit import Circuit
import numpy as np
from mindquantum.core.gates import H,CNOT,RX,RZ
from mindquantum.core.circuit import dagger

# 第一步，准备量子线路
## TwoLocal-Circular
127 qubits and 508 parameters

三层，每一层包括：每个qubit上一个RX门，最后一个qubit和第一个qubit上一个CNOT门，相邻两个qubits上一个CNOT门

最后每个qubit上一个RX门（详见下侧ansatz线路示例）

In [2]:
def build_ansatz(n_qubits,depth):
    circ = Circuit()
    for i in range(depth):
        for j in range(n_qubits):
            circ += RX(f'theta{i*n_qubits+j}').on(j)
        circ += CNOT.on(0,n_qubits-1)
        for j in range(n_qubits-1):
            circ += CNOT.on(j+1,j)
    for j in range(n_qubits):
        circ += RX(f'theta{depth*n_qubits+j}').on(j)
    return circ

In [3]:
# 一层ansatz线路示例
build_ansatz(3,1).svg()

## 编译线路
假设编译所用量子门集只包含H，CNOT，RZ门

RX门编译后变成$H*RZ*H$

In [4]:
# 编译后的电路只含有H，CNOT，RZ门
def compile_circuit(circ):
    circ_compiled = Circuit()
    for gate in circ:
        if gate.name == 'H' or gate.name == 'CNOT' or gate.name == 'RZ':
            circ_compiled += gate
        elif gate.name == 'RX':
            circ_compiled += H.on(gate.obj_qubits)
            circ_compiled += RZ(gate.coeff).on(gate.obj_qubits)
            circ_compiled += H.on(gate.obj_qubits)

    return circ_compiled

In [5]:
# 一层ansatz线路编译后生成的线路
compile_circuit(build_ansatz(3,1)).svg()

In [6]:
# 完整线路共3层127个量子比特
n_qubits = 127
depth = 3
circ1 = build_ansatz(n_qubits,depth)
circ1.summary()

==============================================Circuit Summary==============================================
|Total number of gates  : 889.                                                                            |
|Parameter gates        : 508.                                                                            |
|with 508 parameters are:                                                                                 |
|theta0, theta1, theta2, theta3, theta4, theta5, theta6, theta7, theta8, theta9..                        .|
|Number qubit of circuit: 127                                                                             |


In [7]:
circ1_inv = dagger(circ1)    # dagger将量子线路左右逆转
circ2 = compile_circuit(circ1)
circ2.summary()
circ_all = circ1_inv + circ2
# circ_all.svg()

==============================================Circuit Summary==============================================
|Total number of gates  : 1905.                                                                           |
|Parameter gates        : 508.                                                                            |
|with 508 parameters are:                                                                                 |
|theta0, theta1, theta2, theta3, theta4, theta5, theta6, theta7, theta8, theta9..                        .|
|Number qubit of circuit: 127                                                                             |


# 第二步，将量子线路转换成ZX图
## ZX图
用ZX图表示量子线路，量子门是ZX图中的顶点，将顶点涂成3种颜色：

yellow：H门

red：X-spider，RX门，CNOT门的受控位

green：Z-spider，RZ门，CNOT门的控制位


In [8]:
# 顶点
class Vertex:
    def __init__(self, name, color,qubit, neighbor, phase=0.0):
        self.name = name    # 量子门序号
        self.color = color
        self.phase = phase   # 量子门相位
        self.qubit = qubit     # 作用在哪个量子比特上
        self.neighbor = neighbor    # 前一个量子门、后一个量子门、存在控制关系的量子门的序号

In [9]:
# ZX图默认无环，省略了ZX演算的无环规则（本文中，所有“无环”均指不存在单条边构成的环）
class Graph:
    def __init__(self):
        self.vertices = {}    # 初始图，空
        self.count = 0       # 顶点总数，只增不减，也用于给新顶点命名

    # 新增边
    def add_edge(self, from_vertex, to_vertex):
        self.vertices[from_vertex].neighbor.append(to_vertex)

    # 新增顶点
    def add_vertex(self, color, qubit, neighbor, phase=0.0):
        name = self.count
        self.count += 1
        self.vertices[name] = Vertex(name, color, qubit, neighbor, phase)    # 新增顶点时已在图中绘制了与相邻顶点的单向边
        for v in neighbor:    # 再绘制反方向的边
            self.add_edge(v, name)

    # 打印图信息
    def print(self):
        print("==================graph message==================")
        for v in self.vertices.values():
            print(v.name, '\t', v.neighbor, '\t', v.color, '\t', v.phase)
        print('\n')

    # 删除自身的环
    def clear(self):
        for v in self.vertices.values():
            while v.name in v.neighbor:
                self.vertices[v.name].neighbor.remove(v.name)

    # 删除顶点
    def delete_vertex(self, name):
        for v in self.vertices.values():
            while name in v.neighbor:    # 删除与该顶点有关的所有边（此处删除终点是该顶点的边）
                self.vertices[v.name].neighbor.remove(name)
        self.vertices.pop(name)    # 删除该顶点（此处删除起点是该顶点的边）

    # 两个电路是否等价
    def equiv(self):
        if len(self.vertices) == 0:    # 等价的两个电路，经过ZX化简后，图中无顶点
            print("Equivalent!")
        else:
            print("Not sure!")

In [10]:
# 将量子线路绘制成ZX图
def draw_graph(circ):
    g = Graph()
    # last_name保存每个qubit上当前的最后一个顶点的name
    last_name = [-1] * circ.n_qubits
    for gate in circ:
        if gate.name == 'H':
            # 当前qubit上已有顶点
            if last_name[gate.obj_qubits[0]] != -1:
                g.add_vertex('yellow', gate.obj_qubits[0], [last_name[gate.obj_qubits[0]]])
            # 当前qubit上暂无顶点
            else:
                g.add_vertex('yellow', gate.obj_qubits[0], [])
            # 更新当前qubit上最后一个顶点的name为新增顶点
            last_name[gate.obj_qubits[0]] = g.count-1
        if gate.name == 'RX':
            if last_name[gate.obj_qubits[0]] != -1:
                # 顶点phase = RX门参数
                g.add_vertex('red', gate.obj_qubits[0], [last_name[gate.obj_qubits[0]]], gate.coeff)
            else:
                g.add_vertex('red', gate.obj_qubits[0], [], gate.coeff)
            last_name[gate.obj_qubits[0]] = g.count-1
        if gate.name == 'RZ':
            if last_name[gate.obj_qubits[0]] != -1:
                g.add_vertex('green', gate.obj_qubits[0], [last_name[gate.obj_qubits[0]]], gate.coeff)
            else:
                g.add_vertex('green', gate.obj_qubits[0], [], gate.coeff)
            last_name[gate.obj_qubits[0]] = g.count-1
        if gate.name == 'CNOT':
            # 控制位顶点
            if last_name[gate.obj_qubits[1]] != -1:
                g.add_vertex('green', gate.obj_qubits[1], [last_name[gate.obj_qubits[1]]])
            else:
                g.add_vertex('green', gate.obj_qubits[1], [])
            last_name[gate.obj_qubits[1]] = g.count-1
            # 受控位顶点
            if last_name[gate.obj_qubits[0]] != -1:
                # 受控顶点neighbor中默认包含控制位顶点
                g.add_vertex('red', gate.obj_qubits[0], [last_name[gate.obj_qubits[0]],  g.count-1])
            else:
                g.add_vertex('red', gate.obj_qubits[0], [g.count-1])
            last_name[gate.obj_qubits[0]] = g.count-1
    return g

In [11]:
g = draw_graph(circ_all)
# g.print()

# 第三步，化简ZX图
利用ZX演算规则进行化简，如果化简直接得到了线路等价的结果，则终止；否则，继续下一步。

## ZX演算规则
## rule_f

<img src="picture/equivalence_checking_of_PQC_rule_f.jpg" width="25%"> 

In [12]:
def rule_f(g:Graph):
    # ZX演算过程中，图中的顶点会发生增减
    # 获取最初的所有顶点
    for v1 in list(g.vertices.keys()):
        # 判断当前顶点在化简过程中有没有被删除
        if v1 not in g.vertices.keys():
            continue
        v1 = g.vertices[v1]
        if v1.color == 'red' or v1.color == 'green':    # 不化简黄色顶点
            for v2 in v1.neighbor:
                v2 = g.vertices[v2]
                if v2.color == v1.color:    # 相同颜色
                    v2.phase += v1.phase
                    v2.neighbor.extend(v1.neighbor)    # 合并两个顶点
                    for v3 in v1.neighbor:
                        v3 = g.vertices[v3]
                        v3.neighbor.append(v2.name)
                    g.delete_vertex(v1.name)    # 删除已被合并的顶点
                    g.clear()    # 清除过程中可能产生的环

## rule_id

<img src="picture/equivalence_checking_of_PQC_rule_id.jpg" width="30%"> 

In [13]:
def rule_id(g:Graph):
    for v1 in list(g.vertices.keys()):
        if v1 not in g.vertices.keys():
            continue
        v1 = g.vertices[v1]
        if v1.phase == 0 or list(v1.phase.values()) == [0.0]*len(list(v1.phase.values())):    # phase =0（phase可能是列表）
            flag = True
            for v2 in v1.neighbor:
                v2 = g.vertices[v2]
                if v2.qubit != v1.qubit:    # 与其他量子位上的顶点相关
                    flag = False
            if flag:
                for v2 in v1.neighbor:
                    v2 = g.vertices[v2]
                    v2.neighbor.extend(v1.neighbor)    # 将前一个顶点与后一个顶点相连，略过当前顶点
                g.delete_vertex(v1.name)
                g.clear()

## rule_hh

<img src="picture/equivalence_checking_of_PQC_rule_hh.jpg" width="30%"> 

In [14]:
def rule_hh(g:Graph):
    for v1 in list(g.vertices.keys()):
        if v1 not in g.vertices.keys():
            continue
        v1 = g.vertices[v1]
        if v1.color == 'yellow':
            for v2 in v1.neighbor:
                v2 = g.vertices[v2]
                if v2.qubit != v1.qubit:    # 当前顶点与其他qubit上的顶点相关
                    return
            for v2 in v1.neighbor:
                v2 = g.vertices[v2]
                if v2.color == 'yellow':    # 都是yellow顶点
                    v1.neighbor.remove(v2.name)    # 最左边和最右边相连，略过两个yellow顶点
                    v2.neighbor.remove(v1.name)
                    for v3 in v1.neighbor:
                        v3 = g.vertices[v3]
                        v3.neighbor.extend(v2.neighbor)
                    for v4 in v2.neighbor:
                        v4 = g.vertices[v4]
                        v4.neighbor.extend(v1.neighbor)
                    g.delete_vertex(v1.name)
                    g.delete_vertex(v2.name)
                    g.clear()

## rule_h

<img src="picture/equivalence_checking_of_PQC_rule_h.jpg" width="40%"> 

In [15]:
def rule_h(g:Graph):
    for v1 in list(g.vertices.keys()):
        if v1 not in g.vertices.keys():
            continue
        v1 = g.vertices[v1]
        if v1.color == 'green':
            flag = True
            for v2 in v1.neighbor:
                v2 = g.vertices[v2]
                if v2.color != 'yellow':    # 不满足所有邻居都是yellow
                    flag = False
                    break
            if flag:
                v1.color = 'red'    # 当前顶点从green变成red
                for v2 in list(v1.neighbor):    # 顶点的所有邻居都是yellow，略过并删除这些yellow顶点
                    v2 = g.vertices[v2]
                    v2.neighbor.remove(v1.name)
                    v1.neighbor.extend(v2.neighbor)
                    for v3 in v2.neighbor:
                        v3 = g.vertices[v3]
                        v3.neighbor.append(v1.name)
                    g.delete_vertex(v2.name)
                    g.clear()

## rule_b

<img src="picture/equivalence_checking_of_PQC_rule_b.jpg" width="30%"> 

In [16]:
def rule_b(g:Graph):
    for v1 in list(g.vertices.keys()):
        if v1 not in g.vertices.keys():
            continue
        v1 = g.vertices[v1]
        if v1.color == 'green':    # 沙漏左上角
            for v2 in v1.neighbor:
                v2 = g.vertices[v2]
                if v2.color == 'red' and v2.qubit == v1.qubit:    # 沙漏右上角
                    for v3 in v2.neighbor:
                        v3 = g.vertices[v3]
                        if v3.color == 'green' and v3.qubit != v2.qubit:    # 沙漏左下角
                            for v4 in v3.neighbor:
                                v4 = g.vertices[v4]
                                if v4.color == 'red' and v4.qubit == v3.qubit and v1.name in v4.neighbor:    # 沙漏右下角
                                    v1.color = 'red'    # 反转颜色，并将沙漏左侧合二为一
                                    v3.neighbor.remove(v2.name)
                                    v3.neighbor.remove(v4.name)
                                    v1.neighbor.extend(v3.neighbor)
                                    v2.color = 'green'    # 反转颜色，并将沙漏右侧合二为一
                                    v4.neighbor.remove(v1.name)
                                    v4.neighbor.remove(v3.name)
                                    v2.neighbor.extend(v4.neighbor)
                                    for v in v3.neighbor:
                                        v = g.vertices[v]
                                        v.neighbor.append(v1.name)
                                    for v in v4.neighbor:
                                        v = g.vertices[v]
                                        v.neighbor.append(v2.name)
                                    g.delete_vertex(v3.name)
                                    g.delete_vertex(v4.name)
                                    g.clear()

## rule_1

<img src="picture/equivalence_checking_of_PQC_rule_1.jpg" width="40%"> 

In [17]:
def rule_1(g:Graph):
    for v1 in list(g.vertices.keys()):
        if v1 not in g.vertices.keys():
            continue
        v1 = g.vertices[v1]
        if v1.color == 'green':
            for v2 in v1.neighbor:
                v2 = g.vertices[v2]
                if v2.color == 'red' and v2.neighbor.count(v1.name) == 2:    # 红绿顶点，且两顶点间有两条边
                    while v2.name in g.vertices[v1.name].neighbor:    # 删除相连的边
                        v1.neighbor.remove(v2.name)
                    while v1.name in g.vertices[v2.name].neighbor:
                        v2.neighbor.remove(v1.name)

## rule_2

<img src="picture/equivalence_checking_of_PQC_rule_2.jpg" width="45%"> 

In [18]:
def rule_2(g:Graph):
    for v1 in list(g.vertices.keys()):
        if v1 not in g.vertices.keys():
            continue
        v1 = g.vertices[v1]
        if v1.color== 'red':    # 上一
            for v2 in v1.neighbor:
                v2 = g.vertices[v2]
                if v2.color == 'green' and v2.qubit == v1.qubit:    # 上二
                    for v3 in v2.neighbor:
                        v3 = g.vertices[v3]
                        if v3.color == 'red' and v3.qubit == v2.qubit and v3.name != v1.name:    # 上三
                            for v6 in v3.neighbor:
                                v6 = g.vertices[v6]
                                if v6.color == 'green' and v6.qubit != v3.qubit:    # 下三
                                    for v5 in v6.neighbor:
                                        v5 = g.vertices[v5]
                                        if v5.color == 'red' and v5.qubit == v6.qubit and v2.name in v5.neighbor:    # 下二
                                            for v4 in v5.neighbor:
                                                v4 = g.vertices[v4]
                                                if v4.color == 'green' and v4.qubit == v5.qubit and v1.name in v4.neighbor:    # 下一
                                                    g.delete_vertex(v2.name)    # 全部删除
                                                    g.delete_vertex(v5.name)
                                                    v1.neighbor.remove(v4.name)
                                                    v4.neighbor.remove(v1.name)
                                                    v3.neighbor.remove(v6.name)
                                                    v6.neighbor.remove(v3.name)
                                                    for v in v1.neighbor:
                                                        v = g.vertices[v]
                                                        v.neighbor.extend(v6.neighbor)
                                                    for v in v6.neighbor:
                                                        v = g.vertices[v]
                                                        v.neighbor.extend(v1.neighbor)
                                                    for v in v4.neighbor:
                                                        v = g.vertices[v]
                                                        v.neighbor.extend(v3.neighbor)
                                                    for v in v3.neighbor:
                                                        v = g.vertices[v]
                                                        v.neighbor.extend(v4.neighbor)
                                                    g.delete_vertex(v1.name)
                                                    g.delete_vertex(v3.name)
                                                    g.delete_vertex(v4.name)
                                                    g.delete_vertex(v6.name)
                                                    g.clear()
                                                    for v in g.vertices.values():    # 化简后只剩交叉
                                                        if v.qubit == v3.qubit and v.name > v3.name:
                                                            v.qubit = v6.qubit
                                                        elif v.qubit == v6.qubit and v.name > v6.name:
                                                            v.qubit = v3.qubit

## rule_neg

<img src="picture/equivalence_checking_of_PQC_rule_neg.jpg" width="35%"> 

In [19]:
def rule_neg(g:Graph):
    for v1 in list(g.vertices.keys()):
        if v1 not in g.vertices.keys():
            continue
        v1 = g.vertices[v1]
        if v1.phase != 0:
            for v2 in v1.neighbor:
                v2 = g.vertices[v2]
                if v2.color == v1.color and v2.phase == -v1.phase:    # 相同颜色、相反相位
                    v2.phase = 0
                    v2.neighbor.extend(v1.neighbor)
                    for v3 in v1.neighbor:
                        v3 = g.vertices[v3]
                        v3.neighbor.append(v2.name)
                    g.delete_vertex(v1.name)
                    g.clear()
                    rule_id(g)

In [20]:
# 利用以上规则进行化简
def simplify(g : Graph):
    temp = []
    while(temp != list(g.vertices.keys())):
        temp = list(g.vertices.keys())    # temp用于对比本轮循环是否有效化简
        rule_hh(g)
        rule_h(g)
        rule_neg(g)
        rule_b(g)
        rule_2(g)
        rule_f(g)
        rule_1(g)
        rule_id(g)

In [21]:
# 化简TwoLocal-Circular
print("化简之前：")
g.equiv()

simplify(g)

print("化简之后：")
g.equiv()

化简之前：
Not sure!
化简之后：
Equivalent!


# 第四步，若ZX演算无法确定等价性，则实例化参数
判断实例化之后的“无参数”线路是否等价，如果得到线路不等价的结果，则终止；否则，重复本步骤若干次，最后一次尝试随机实例化参数，这次的结果就是最终结果。

## 反例：ZX演算化简后无法确定、实际上不等价的线路

In [22]:
neq_circ1 = Circuit()
neq_circ1 += H.on(1)
neq_circ1 += RX(f'theta{0}').on(2)
neq_circ1 += CNOT.on(0,1)
neq_circ1 += RZ(f'theta{1}').on(0)
neq_circ1 += CNOT.on(2,1)
neq_circ1 += CNOT.on(0,1)
neq_circ1 += RX(f'theta{2}').on(2)

neq_circ1.svg()

In [23]:
neq_circ2 = Circuit()
neq_circ2 += H.on(1)
neq_circ2 += RX(f'theta{0}').on(2)
neq_circ2 += CNOT.on(0,1)
neq_circ2 += RZ(f'theta{1}').on(0)
neq_circ2 += CNOT.on(2,1)
neq_circ2 += CNOT.on(0,1)
# 不同之处
neq_circ2 += RX({f'theta{0}': 1, f'theta{1}': 1, f'theta{2}': 1}).on(2)

neq_circ2.svg()

In [24]:
neq_circ1_inv = dagger(neq_circ1)
neq_circ_all = neq_circ1_inv + neq_circ2
neq_circ_all.svg()

In [25]:
neq_g = draw_graph(neq_circ_all)
print("化简之前：")
neq_g.equiv()

simplify(neq_g)

print("化简之后：")
neq_g.equiv()

化简之前：
Not sure!
化简之后：
Not sure!


## ZX演算无法确定，于是需要实例化参数进行验证

In [26]:
# 映射函数实例化参数
def map_para(n,r):
    para = {}
    for i in range(n):
        para[f'theta{i}'] = (2*np.pi/((i+1)*r)-np.pi)
    return para

In [27]:
# 随机实例化参数
def random_para(n):
    para = {}
    for i in range(n):
        para[f'theta{i}'] = (np.random.uniform(np.pi,-np.pi))
    return para

In [28]:
# 实例化参数验证circ1和circ2，验证r轮
def verify_by_para(circ1,circ2,r):
    # 线路中一共n个参数
    n = len(list(set(circ1.params_name+circ2.params_name)))
    # 记录前r-1轮验证是否有结果
    flag = True
    # 前r-1轮指定参数
    for i in range(r-1):
        para = map_para(n,i+1)

        if np.array_equal(circ1.matrix(para), circ2.matrix(para)):
            continue
        else:
            print('Not equivalent!')
            flag = False
            break

    # 最后一轮随机参数
    if flag:
        para = random_para(n)
        if np.array_equal(circ1.matrix(para), circ2.matrix(para)):
            print('Equivalent!')
        else:
            print('Not equivalent!')

In [29]:
# 实例化参数之后，验证发现线路不等价
verify_by_para(neq_circ1,neq_circ2,5)

Not equivalent!


# 最后，将以上过程合并成一个完整的功能

In [30]:
def ZXcalculus (circ1, circ2):
    circ1_inv = dagger(circ1)
    circ = circ1_inv + circ2
    g = draw_graph(circ)
    print("化简之前：")
    g.equiv()
    simplify(g)
    print("化简之后：")
    if len(g.vertices) == 0:
        g.equiv()
    else:
        g.equiv()
        print("实例化参数验证：")
        verify_by_para(circ1,circ2,5)

In [31]:
# 验证ZX-calculus功能
# 由于线路中的参数已经进行过变换，所以要重新构造
circ1 = build_ansatz(n_qubits,depth)
circ2 = compile_circuit(circ1)

ZXcalculus(circ1,circ2)

化简之前：
Not sure!
化简之后：
Equivalent!


In [32]:
neq_circ1 = Circuit()
neq_circ1 += H.on(1)
neq_circ1 += RX(f'theta{0}').on(2)
neq_circ1 += CNOT.on(0,1)
neq_circ1 += RZ(f'theta{1}').on(0)
neq_circ1 += CNOT.on(2,1)
neq_circ1 += CNOT.on(0,1)
neq_circ1 += RX(f'theta{2}').on(2)

neq_circ2 = Circuit()
neq_circ2 += H.on(1)
neq_circ2 += RX(f'theta{0}').on(2)
neq_circ2 += CNOT.on(0,1)
neq_circ2 += RZ(f'theta{1}').on(0)
neq_circ2 += CNOT.on(2,1)
neq_circ2 += CNOT.on(0,1)
# 不同之处
neq_circ2 += RX({f'theta{0}': 1, f'theta{1}': 1, f'theta{2}': 1}).on(2)

ZXcalculus(neq_circ1,neq_circ2)

化简之前：
Not sure!
化简之后：
Not sure!
实例化参数验证：
Not equivalent!
